<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Mohammad Abolnejadian
### Student Number : 98103867
### Colab Link: 
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [1]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

/content/dataset.zip: No such file or directory


In [2]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

mkdir: /content: No such file or directory
unzip:  cannot find or open /content/dataset.zip, /content/dataset.zip.zip or /content/dataset.zip.ZIP.


# Preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [2]:
df = pd.read_csv('content/news-train.csv')
df

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...
...,...,...
197356,sports,به گزارش ورزش سه و به نقل از مارکا رقابت‌های ب...
197357,sports,به گزارش ایسنا و به نقل از فوتبال ایتالیا یوون...
197358,sports,سرمربی تیم والیبال دانشجویان گفت ما اصلا فرصت ...
197359,sports,به گزارش ورزش سه مرتضی # باشگاه # مصر # را نسب...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [3]:
topic_count = df.groupby(['topic']).count()
total_number_of_news = topic_count['text'].sum()
print("Total number of news: " + str(total_number_of_news))
topic_count

Total number of news: 197361


,text
topic,
cultural,13359
economics,49600
politics,11297
sports,123105


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [4]:
downsampled_df = pd.DataFrame()
min_number_of_news_for_topic = topic_count['text'].min()

for topic in topic_count.index:
  data = df[df['topic'] == topic]
  downsampled_data = data.sample(min_number_of_news_for_topic)
  downsampled_df = pd.concat([downsampled_df, downsampled_data])

print(downsampled_df)

print("########")

downsampled_topic_count = downsampled_df.groupby(['topic']).count()
print(downsampled_topic_count)

           topic                                               text
13326   cultural  وزیر علوم تحقیقات و فناوری در جمع نمایندگان شو...
161606  cultural  به گزارش ایسنا بر اساس خبر رسیده از خانه کتاب ...
193127  cultural  به گزارش ایسنا این # اظهار نظر یکی از اعضای کم...
127412  cultural  به گزارش ایسنا پژمان محمدی در نشست خبری با اصح...
2497    cultural  وزیر علوم تحقیقات و فناوری گفت مجموعه سمت با م...
...          ...                                                ...
79101     sports  به گزارش ورزش سه از بین بازیکنان استقلال که هف...
66545     sports  به گزارش ورزش سه اتلتیکو مادرید شب گذشته سومین...
142481    sports  فریدون # پس از قبول شکست تیمش برابر ذوب آهن اص...
55770     sports  به گزارش ایسنا و به نقل از فوتبال ایتالیا تا پ...
118700    sports  به‌ گزارش ‌ایسنا ‌و ‌به نقل از # روند # اینتر ...

[45188 rows x 2 columns]
########
            text
topic           
cultural   11297
economics  11297
politics   11297
sports     11297


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [5]:
train_df = downsampled_df.sample(frac=0.8, random_state=1)
test_df = downsampled_df.drop(train_df.index)
print("train df: ")
print(train_df)
print("test_df: ")
print(test_df)

train df: 
            topic                                               text
194423     sports  به گزارش ورزش سه شاید خیلی‌ها # # چطور توانسته...
127577     sports  به گزارش ورزش سه کارلوس کی‌روش قصد ندارد به مو...
3486     politics  علی نادری فعال رسانه‌ای رهبر انقلاب در دیدار #...
26185   economics  به گزارش ایسنا # گندم و واقعی بودن و # بودن آن...
31260      sports  به گزارش ورزش سه پرسپولیسی‌ها که از هفته گذشته...
...           ...                                                ...
136905     sports  سرمربی ایتالیایی # می‌گوید که تیمش در دیدار بر...
9840    economics  علیرغم پیشنهاد # برای ادامه یارانه شیر به صورت...
140492     sports  به گزارش ورزش سه با توجه به توقف چلسی در دیدار...
184622   politics  علی مطهری نماینده تهران در مجلس شورای اسلامی گ...
91170   economics  نمایندگان مجلس شورای اسلامی در حالی فعالیت خود...

[36150 rows x 2 columns]
test_df: 
           topic                                               text
176111  cultural  این روزها اهالی تجسمی به دلیل فروش بالا

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [6]:
train_df['tokens'] = train_df['text'].str.strip().str.split(' ')
train_df

,topic,text,tokens
194423,sports,به گزارش ورزش سه شاید خیلی‌ها # # چطور توانسته...,"[به, گزارش, ورزش, سه, شاید, خیلی‌ها, #, #, چطو..."
127577,sports,به گزارش ورزش سه کارلوس کی‌روش قصد ندارد به مو...,"[به, گزارش, ورزش, سه, کارلوس, کی‌روش, قصد, ندا..."
3486,politics,علی نادری فعال رسانه‌ای رهبر انقلاب در دیدار #...,"[علی, نادری, فعال, رسانه‌ای, رهبر, انقلاب, در,..."
26185,economics,به گزارش ایسنا # گندم و واقعی بودن و # بودن آن...,"[به, گزارش, ایسنا, #, گندم, و, واقعی, بودن, و,..."
31260,sports,به گزارش ورزش سه پرسپولیسی‌ها که از هفته گذشته...,"[به, گزارش, ورزش, سه, پرسپولیسی‌ها, که, از, هف..."
...,...,...,...
136905,sports,سرمربی ایتالیایی # می‌گوید که تیمش در دیدار بر...,"[سرمربی, ایتالیایی, #, می‌گوید, که, تیمش, در, ..."
9840,economics,علیرغم پیشنهاد # برای ادامه یارانه شیر به صورت...,"[علیرغم, پیشنهاد, #, برای, ادامه, یارانه, شیر,..."
140492,sports,به گزارش ورزش سه با توجه به توقف چلسی در دیدار...,"[به, گزارش, ورزش, سه, با, توجه, به, توقف, چلسی..."
184622,politics,علی مطهری نماینده تهران در مجلس شورای اسلامی گ...,"[علی, مطهری, نماینده, تهران, در, مجلس, شورای, ..."


Count the tokens and list the top 5 frequent ones (5 points)

In [7]:
train_df_exploded = train_df.explode('tokens')
top_5 = train_df_exploded['tokens'].value_counts().nlargest(5)
top_5_df = pd.DataFrame({'token': top_5.index, 'frequency': top_5.values})
top_5_df

,token,frequency
0,#,1616849
1,و,873470
2,در,741159
3,\n,586159
4,به,579760


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [ ]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-06 18:55:58--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-06 18:55:58--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-06 18:55:58 (43.7 M

In [8]:
#read stopwords
stopwords_file = 'stopwords.txt'
with open(stopwords_file) as f:
    stopwords = set(f.read().splitlines())
stopwords.update(['#', '\n'])
#remove from text
train_df['text'] = train_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#remove from tokens
train_df['tokens'] = train_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])
train_df

,topic,text,tokens
194423,sports,گزارش ورزش سه شاید خیلی‌ها توانسته پله پله راه...,"[گزارش, ورزش, سه, شاید, خیلی‌ها, توانسته, پله,..."
127577,sports,گزارش ورزش سه کارلوس کی‌روش قصد موضوع چالش برا...,"[گزارش, ورزش, سه, کارلوس, کی‌روش, قصد, موضوع, ..."
3486,politics,علی نادری فعال رسانه‌ای رهبر انقلاب دیدار اعضا...,"[علی, نادری, فعال, رسانه‌ای, رهبر, انقلاب, دید..."
26185,economics,گزارش ایسنا گندم واقعی محمدرضا وزیر اسبق جهاد ...,"[گزارش, ایسنا, گندم, واقعی, محمدرضا, وزیر, اسب..."
31260,sports,گزارش ورزش سه پرسپولیسی‌ها هفته گذشته اردوی دو...,"[گزارش, ورزش, سه, پرسپولیسی‌ها, هفته, گذشته, ا..."
...,...,...,...
136905,sports,سرمربی ایتالیایی می‌گوید تیمش دیدار برابر رئال...,"[سرمربی, ایتالیایی, می‌گوید, تیمش, دیدار, براب..."
9840,economics,علیرغم پیشنهاد برای ادامه یارانه شیر غیر نقدی ...,"[علیرغم, پیشنهاد, برای, ادامه, یارانه, شیر, غی..."
140492,sports,گزارش ورزش سه توجه توقف چلسی دیدار عصر برایتون...,"[گزارش, ورزش, سه, توجه, توقف, چلسی, دیدار, عصر..."
184622,politics,علی مطهری نماینده تهران مجلس شورای اسلامی سال‌...,"[علی, مطهری, نماینده, تهران, مجلس, شورای, اسلا..."


Make a list of the remaining unique tokens (5 points)

In [9]:
train_df_exploded = train_df.explode('tokens')
unique_tokens = train_df_exploded['tokens'].unique()
number_of_unique_tokens = unique_tokens.size
print(unique_tokens.tolist())
print("number of unique tokens: " + str(number_of_unique_tokens))

['گزارش', 'ورزش', 'سه', 'شاید', 'خیلی\u200cها', 'توانسته', 'پله', 'راه\u200cهای', 'طی', 'کند', 'نهایت', 'باشگاه', 'استقلال', 'برسد', 'حرف\u200cهای', 'برنامه', 'فوتبال', 'برتر', 'سبب', 'ابعاد', 'تازه\u200cای', 'زندگی', 'برای', 'مخاطبان', 'آشکار', 'این', 'بازیکن', 'نسبت', 'گذشته', 'محبوبیت', 'بیشتری', 'پیدا', 'تصویر', 'صفحه', 'فارسی', 'اینستاگرام', 'کنفدراسیون', 'آسیا', 'منتشر', 'روزهایی', 'می\u200cدهد', 'دوران', 'کودکی', 'پیراهن', 'تیم', 'شاهین', 'بوشهر', 'بخشی', 'زمین\u200cهای', 'سپری', 'کارلوس', 'کی\u200cروش', 'قصد', 'موضوع', 'چالش', 'برانکو', 'ایوانکوویچ', 'پایان', 'شرایطی', 'شب', 'ملی', 'یمن', 'برتری', 'گله', 'ایران', 'همراه', 'هواداران', 'برد', 'ترجیح', 'متوجه', 'تاثیر', 'قرار', 'گیرد', 'سرمربی', 'انتقاد', 'صحبت\u200cهای', 'عراق', 'کروات', 'همچنین', 'نقل', 'انتقالات', 'بابت', 'صعود', 'فینال', 'رخ', 'توجه', 'پرسپولیس', 'کرد', 'رابطه', 'اینکه', 'ایرج', 'عرب', 'سرپرست', 'اعلام', 'احترام', 'پاسخی', 'صحبت\u200cها', 'مدیر', 'روابط', 'عمومی', 'نیز', 'حال', 'حاضر', 'امیدواری', 'حریفان', 'ق

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [10]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix = vectorizer.fit_transform(train_df['text'])
x_train = term_doc_matrix.toarray()
x_train

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 3, ..., 0, 0, 1],
       [0, 0, 3, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]])

In [ ]:
###Custom solution to this problem adding a new column to the datafram

# from collections import Counter

# # def count_occurrences(lst):
# #     return {i: lst.count(i) for i in lst}

# train_df['occ'] = train_df['tokens'].apply(lambda x: dict(Counter(x)))


# train_df['x_train'] = train_df['occ'].apply(lambda x: [x.get(token, 0) for token in unique_tokens])
# # df = df.drop(['new_col_1', ], axis=1, errors='ignore')
# train_df = train_df.drop('occ', axis=1)
# train_df

In [ ]:
#Check the sanity of our solution
# for token in unique_tokens[:10]:
#   print(token)
# Counter(train_df.at[165543, 'tokens'])

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [11]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_df['topic'])
y_train

array([3, 3, 2, ..., 3, 2, 1])

In [11]:
#Another way with pandas factorize
# train_df['y_train'] = pd.factorize(train_df['topic'])[0]
# train_df
# x_train = train_df['x_train'].values
# y_train = train_df['y_train'].values
# x_train.size

array([3, 3, 2, ..., 3, 2, 1])

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [12]:
###preprocess test data
#tokenize
test_df['tokens'] = test_df['text'].str.strip().str.split(' ')
#remove stopwords from text
test_df['text'] = test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
#remove stopwords from tokens
test_df['tokens'] = test_df['tokens'].apply(lambda x: [word for word in x if word not in stopwords])

In [13]:
vectorizer = CountVectorizer(vocabulary=unique_tokens)
term_doc_matrix = vectorizer.fit_transform(test_df['text'])
x_test = term_doc_matrix.toarray()
x_test

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [ ]:
###generate x_test(custom)
# test_df['occ'] = test_df['tokens'].apply(lambda x: dict(Counter(x)))
# test_df['x_test'] = test_df['occ'].apply(lambda x: [x.get(token, 0) for token in unique_tokens])
# test_df = test_df.drop('occ', axis=1)
# test_df

#iterate through rows and add x_train
# default_list = [0] * number_of_unique_tokens
# test_df['x_test'] = pd.Series([list(default_list) for x in range(len(df.index))])
# for index, row in test_df.iterrows():
#     # row['x_train'] = [0] * number_of_unique_tokens
#     token_occurences = dict(Counter(row['tokens']))
#     for token, occ in token_occurences.items():
#       tokens_idx = mapping_token_to_index[token]
#       row['x_test'][tokens_idx] = occ
# test_df

In [14]:
encoder = LabelEncoder()
y_test = encoder.fit_transform(test_df['topic'])
y_test

array([0, 0, 0, ..., 3, 3, 3])

In [63]:
###Turning columns into numpy arrays (in the custom solution)
# x_test = test_df['x_test'].values
# y_test = test_df['y_test'].values
# x_test.size

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [15]:
clf = MultinomialNB(force_alpha=True)
clf.fit(x_train, y_train)

MultinomialNB(force_alpha=True)

Get the predictions of the model for the testing news (`x_test`).

In [16]:
sklearn_predictions = clf.predict(x_test)
sklearn_predictions

array([0, 0, 0, ..., 3, 3, 3])

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [17]:
print(classification_report(y_test, sklearn_predictions))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      2246
           1       0.91      0.94      0.93      2198
           2       0.95      0.91      0.93      2280
           3       0.99      0.98      0.99      2314

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [84]:
#We use gaussian naive bayes
import math
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        """
        YOUR CODE (IF NECESSARY)
        """
        self.classes = None
        self.number_of_classes = None
        self.priors = dict()
        self.mean = dict()
        self.var = dict()
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        #calculate initial values
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        self.number_of_classes = len(self.classes)
        
        for c in self.classes:
            x_c = X[y == c]
            self.priors[c] = x_c.shape[0] / float(n_samples)
            self.mean[c] = x_c.mean(axis=0)
            self.var[c] = x_c.var(axis=0)
        
        # self.mean = np.zeros((number_of_classes, n_features), dtype=np.float64)
        # self.var = np.zeros((number_of_classes, n_features), dtype=np.float64)
        # self.priors = np.zeros(number_of_classes, dtype=np.float64)
        
        # for idx, clss in enumerate(self.classes):
        #   x_only_this_class = X[y == clss]
        #   self.priors[idx] = x_only_this_class.shape[0] / float(n_samples)
        #   self.var[idx, :] = x_only_this_class.var(axis=0)
        #   self.mean[idx, :] = x_only_this_class.mean(axis=0)

          
        return self
    
    def normal_pdf(self, c, x):
        eps = 1e-3
        mean = self.mean[c]
        var = self.var[c] + eps
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


    def new_normale_pdf(self, mean, var, x):
        eps = 1e-2 # Added in denominator to prevent division by zero
        coeff = 1.0 / math.sqrt(2.0 * math.pi * var + eps)
        exponent = math.exp(-(math.pow(x - mean, 2) / (2 * var + eps)))
        return coeff * exponent

    def classify(self, x):
      posteriors = []
      for c in self.classes:
        prior = np.log(self.priors[c])
        class_pdfs = np.sum(np.log(self.normal_pdf(c, x)))
        posterior = prior + class_pdfs
        posteriors.append(posterior)

      return self.classes[np.argmax(posteriors)]


    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        y_pred = [self.classify(x) for x in X]
        """
        YOUR CODE
        """
        return y_pred

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [89]:
custom_model = Naive_Bayes()
custom_model.fit(x_train, y_train)

In [46]:
print(type(x_test))

<class 'numpy.ndarray'>


Get the predictions of your model for the testing news (`x_test`).

In [90]:
custom_model_predictions = custom_model.predict(x_test)
custom_model_predictions

/var/folders/26/r6cl1fqx2zz6066l4b3zd3_80000gn/T/ipykernel_86383/521459570.py:79: RuntimeWarning: divide by zero encountered in log
  class_pdfs = np.sum(np.log(self.normal_pdf(c, x)))


[0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [91]:
print(classification_report(y_test, custom_model_predictions))

              precision    recall  f1-score   support

           0       0.69      0.94      0.79      2246
           1       0.85      0.82      0.83      2198
           2       0.91      0.62      0.74      2280
           3       0.89      0.88      0.88      2314

    accuracy                           0.81      9038
   macro avg       0.83      0.81      0.81      9038
weighted avg       0.83      0.81      0.81      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef